# Tipos de datos

Es importante conocer con que tipos de datos estamos trabajando. Los datos pueden ser:

1. Cuantitativos o numéricos (discretos y continuos)
2. Cualitativos o categóricos
3. Ordinales

## Datos cuantitativos 

Son representados por números y son importantes si representan la medida de la cantidad observada 
de cierta característica. Pueden ser:

1. Discretos: solo pueder asuamir un valor de una lista de números específicos, pueden ser contados y listados.

2. Continuos: representan mediciones se suelen redondear a un número fijo de decimales para facilitar su manipulación.

## Datos cualitativos

Estos datos describen categorías no numéricas que representar determinada cualidad. Los datos categóricos pueden tomar valores numéricos (por ejemplo, "0" para indicar "masculino" y "1" para indicar "femenino"), pero esos números no tienen un sentido matemático.

## Datos ordinales

 Es una categoría intermedia entre los cuantitativos y los cualitativos. En estos datos existe un orden (primero, segundo, tercero, etc.) es decir, podemos establecer un ranking, es decir, los datos son cualitativos, pero los números colocados en cada categoría tienen un significado.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

In [ ]:
mainpath = "../datasets/"
filename = "titanic-kaggle/train.csv"
fullpath = os.path.join(mainpath, filename)
data = pd.read_csv (fullpath)
data

In [ ]:
data1 = data

In [ ]:
data1 = data1.drop(['Cabin','Name','Ticket'], axis=1)
data1 = data1[data1['Embarked'].notna()]
data1['Age'] = data1['Age'].fillna(-1)

### Datos ordinales

data1['Embarked'] = data1['Embarked'].map({'Q': 2, 'C': 1, 'S': 0})
data1['Sex'] = data1['Sex'].map({'female': 1, 'male': 0})
data1

In [ ]:
data1.info()

In [ ]:
# Estadistica basica de variables númericas 
data1.describe() 

In [ ]:
"""
Categorización de los datos en hombre/mujeres vivos/muertos
"""
live = data1[(data1["Survived"]==1)]
dead = data1[(data1["Survived"]==0)]
livem = data1[(data1["Survived"]==1) & (data1["Sex"] == 0)]
livef = data1[(data1["Survived"]==1) & (data1["Sex"] == 1)]
deadm = data1[(data1["Survived"]==0) & (data1["Sex"] == 0)]
deadf = data1[(data1["Survived"]==0) & (data1["Sex"] == 1)]
print("De los",data1.shape[0], "pasajeros del titanic","sobrevivieron",live.shape[0],"y murieron",dead.shape[0])
print("De los",live.shape[0], "pasajeros que sobrevivieron",livem.shape[0],"eran hombres y",livef.shape[0],"eran mujeres")
print("De los",dead.shape[0], "pasajeros que murieron",deadm.shape[0],"eran hombres y",deadf.shape[0],"eran mujeres")

In [ ]:
# gráfico de barras de frecuencias relativas.
plt.figure(figsize=(15,5))
plt.subplot(1,4,1)
plot = (100 * data1['Survived'].value_counts() / len(data1['Survived'])).plot(
kind='pie',  autopct='%.2f',title='Sobrevivientes %')
plt.subplot(1,4,2)
plot = (100 * data1['Pclass'].value_counts() / len(data1['Pclass'])).plot(
kind='pie' , autopct='%.2f', title='Pasajeros del Titanic %')
plt.subplot(1,4,3)
plot = (100 * data1['Sex'].value_counts() / len(data1['Sex'])).plot(
kind='pie' , autopct='%.2f', title='Sexo de los Pasajeros del Titanic %')
#plt.subplot(1,4,4)
#plot = (100 * data1['Age'].value_counts() / len(data1['Age'])).plot(
#kind='pie' , autopct='%.2f', title='Edad de los Pasajeros del Titanic %')


# Selección de atributos

Proceso para seleccionar un subconjunto de atributos (columnas) relevantes para la construcción del modelo predictivo. Se utiliza para identificar y eliminar los atributos innecesarios, irrelevantes y/o redundantes que no contribuyen al modelo predictivo o disminuyen su precisión.

Para hacer la selección de atributos, hay que tener en cuenta la relación que existe entre ellos  y así poder eliminarlos de forma individual (univariante) o un un grupo de atributos en forma conjunta (multivariante).

Ejemplo: 

* **Modelo bayesiano**: cada atributo es independiente, por lo tanto, podemos hacer una selección de variable univariante.
* **Modelo de red neuronal**: no asume la independencia de los atributos (usa todos los disponibles) por lo tanto, podemos hacer una selección de variable multivariante.

## Métodos para selección de atributos

1. **Métodos de filtrado**

Aplican una medida estadística para asignar una puntuación a cada atributo. Los atributos luego son clasificados de acuerdo a su puntuación y son, o bien seleccionados para su conservación o eliminados del conjunto de datos. Los métodos de filtrado son a menudo univariantes y consideran a cada atributo en forma independiente, o con respecto a la variable dependiente.

Ejemplos de estos métodos son: coeficientes de correlación de Pearson, prueba de $\chi^2$, prueba de Fisher, ganancia de información.

2. **Métodos empaquetados**

Selección de un conjunto de atributos como un problema de búsqueda, en donde las diferentes combinaciones son evaluadas y comparadas. Para hacer estas evaluaciones se utiliza un modelo predictivo y luego se asigna una puntuación a cada combinación basada en la precisión del modelo.

Ejemplo de este método es el algoritmo de eliminación recursiva (forward, backward) de atributos.

Bibliografía:

https://en.wikipedia.org/wiki/Feature_selection

https://towardsdatascience.com/chi-square-test-for-feature-selection-in-machine-learning-206b1f0b8223



# Método de filtrado

In [ ]:
### Coeficientes de correlación de Pearson
### 
def train_corr(data): 
    correlation = data.corr()
    sns.heatmap(correlation, annot=True, cbar=True, cmap="RdYlGn")

In [ ]:
train_corr(data1)

In [ ]:
# Variable numérica
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 5))
axes = axes.flat
columnas = data1.select_dtypes(include=['float64', 'int']).columns
columnas = columnas.drop('Survived') # objetivo

for i, colum in enumerate(columnas):
    sns.regplot(
        x           = data1[colum],
        y           = data1['Survived'],
        color       = "gray",
        marker      = '.',
        scatter_kws = {"alpha":0.4},
        line_kws    = {"color":"r","alpha":0.7},
        ax          = axes[i]
    )
    axes[i].set_title(f"Sobrevivientes vs {colum}", fontsize = 7, fontweight = "bold")
    axes[i].tick_params(labelsize = 6)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

# Se eliminan los axes vacíos
for i in [8]:
    fig.delaxes(axes[i])
    
fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Correlación con Sobrevivientes', fontsize = 10, fontweight = "bold");

# scikit-learn feature selection

https://scikit-learn.org/stable/modules/feature_selection.html

In [ ]:
from sklearn.feature_selection import SelectKBest #SelectKBest removes all but the highest scoring features
from sklearn.feature_selection import chi2, f_classif #Fisher
from sklearn.feature_selection import RFE # recursive feature elimination
from sklearn.ensemble import ExtraTreesClassifier #decision trees 

### Separamos las columnas objetivo

* Objetivo: y
* Atributos: x

En este caso queremos saber quien sobrevivio

In [ ]:
data1.isnull().any().any()

In [ ]:
data2 = data1

El algoritmo va a aplicar la prueba de Fisher a todos los atributos y va a seleccionar los que mejor resultado obtuvieron.

In [ ]:
x = data2['Survived']
k = 4  # número de atributos a seleccionar
entrenar = data2.drop(['Survived'], axis=1)
columnas = list(entrenar.columns.values)
seleccionadas = SelectKBest(f_classif, k=k).fit(entrenar, x)
atrib = seleccionadas.get_support()
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

# Métodos empaquetados: Eliminación recursiva

Selecciona recursivamente un número cada vez más pequeño de atributos. Primero comienza con todos los atributos del dataset y luego en cada pasada va eliminando aquellos que tenga el menor coeficiente de importancia hasta alcanzar el número de atributos deseado

Es más precisa, pero consume mucho más recursos, ya que requiere entrenar un modelo predictivo.

In [ ]:
modelo = ExtraTreesClassifier() 
erec = RFE(modelo)  
erec = erec.fit(entrenar, x)

In [ ]:
atrib = erec.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

In [ ]:
# Importancia de atributos.
modelo.fit(entrenar, x)
modelo.feature_importances_[:4]

In [ ]:
np.sort(modelo.feature_importances_)[::-1][:4]

# Análisis de componentes principales

Transformación del conjunto original de $m$ variables en otro conjunto de nuevas de $p$ variables ($p<m$) que no esten correlacionadas. 

Conseguir $p$ variables ortogonales nuevas (componentes principales) y cada una de ellas es una combinación lineal de las $m$ variables originales, de forma que expliquen la mayor cantidad de variabilidad del conjunto de datos original posible. 

## Valor y vector propios

Sea $A$ como una matriz de $m\times m$ y $v$ un vector de dimensión $1\times m$ no nulo,entonce , $v$ es un vector propio de $A$ si existe un escalar $c$ tal que:
$
\begin{align}
A v = c v
\end{align}

donde $c$ es el valor propio asociado al vector propio $v$.

Los valores propios de la matriz $A$ se pueden obtener con los polinomios característicos, entonces si:

$
\begin{align}
det (A c - I) = 0
\end{align}
$

los ceros de este polinomio son los valores propios de la matriz $A$.

Para el análisis de componentes principales, cada componente principal es un vector propio de la matriz de covarianzas y la varianza explicada de los datos originales por cada componente principal es el valor propio asociado al vector propio en cuestión. 

Como estamos tratando de recoger la mayor varianza posible, las variables que tengan una magnitud mayor que otras tendrán mayor escala de varianza y, por tanto, dominarían sobre las otras variables, por lo que primero hay que normalizar las variables.




### Escalamiento de las características

Consiste en transformar los datos de entrada (columnas) a una escala común para asegurarse de que cada variable de entrada contribuya de manera equitativa al rendimiento del modelo. 

1. **Escalamiento** (normalización): escala las características de entrada a un rango entre 0 y 1 restando el valor mínimo de la característica a cada punto de datos y dividiendo el resultado por el rango de la característica. Es especialmente útil para datos con distribuciones sesgadas.

$
\begin{align}
X'= \frac{X - X_{min}}{X_{max} -X_{min}}
\end{align}
$

2. **Estandarización**: transforma los datos de entrada de manera que tengan una media de 0 y una desviación estándar de 1, restando la media de la característica a cada punto de datos y dividiendo el resultado por la desviación estándar. La estandarización es beneficiosa cuando se trabaja con algoritmos que asumen que las características de entrada siguen una distribución gaussiana.

$
\begin{align}
X'= \frac{X - \mu}{\sigma}
\end{align}
$

In [ ]:
X = data1.iloc[:, 2:].values # caracteristicas
Y = data1.iloc[:, 1].values # variable a predecir
X, Y

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)#fits the parameters of the data and then transforms it.
#X_test = sc.transform(X)#transforms the data by using parameters already stored in the class
X

In [ ]:
from sklearn.decomposition import PCA

PCA = PCA(n_components=2)
components = PCA.fit_transform(X)
PCA.components_

In [ ]:
cumVar = pd.DataFrame(np.cumsum(PCA.explained_variance_ratio_)*100, 
                      columns=["cumVarPerc"])
expVar = pd.DataFrame(PCA.explained_variance_ratio_*100, columns=["VarPerc"])
pd.concat([expVar, cumVar], axis=1)\
    .rename(index={0: "PC1", 1: "PC2"})